## Importing Required Libraries

In [1]:
from gensim.models import Word2Vec
import pandas as pd
import zipfile
import os
import re
import nltk
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC



## Read the Folder


In [6]:

zip_file_path = 'C:/Users/amnak/Downloads/archive (3).zip'
extraction_directory = '/mnt/data/extracted_files'

os.makedirs(extraction_directory, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_directory)

dataset_folder = os.path.join(extraction_directory, 'Genre Classification Dataset')
description_file = os.path.join(dataset_folder, 'description.txt')
test_data_file = os.path.join(dataset_folder, 'test_data.txt')
test_data_solution_file = os.path.join(dataset_folder, 'test_data_solution.txt')
train_data_file = os.path.join(dataset_folder, 'train_data.txt')

with open(description_file, 'r') as file:
    description_content = file.read()

test_data_df = pd.read_csv(test_data_file, delimiter=' ::: ', engine='python', names=['ID', 'TITLE', 'DESCRIPTION'])
test_data_solution_df = pd.read_csv(test_data_solution_file, delimiter=' ::: ', engine='python', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])
train_data_df = pd.read_csv(train_data_file, delimiter=' ::: ', engine='python', names=['ID', 'TITLE', 'GENRE', 'DESCRIPTION'])

print("\nTest Data:\n", test_data_df.head())
print("\nTrain Data:\n", train_data_df.head())



Test Data:
    ID                        TITLE  \
0   1         Edgar's Lunch (1998)   
1   2     La guerra de papá (1977)   
2   3  Off the Beaten Track (2010)   
3   4       Meu Amigo Hindu (2015)   
4   5            Er nu zhai (1955)   

                                         DESCRIPTION  
0  L.R. Brane loves his life - his car, his apart...  
1  Spain, March 1964: Quico is a very naughty chi...  
2  One year in the life of Albin and his family o...  
3  His father has died, he hasn't spoken with his...  
4  Before he was known internationally as a marti...  

Train Data:
    ID                             TITLE     GENRE  \
0   1      Oscar et la dame rose (2009)     drama   
1   2                      Cupid (1997)  thriller   
2   3  Young, Wild and Wonderful (1980)     adult   
3   4             The Secret Sin (1915)     drama   
4   5            The Unrecovered (2007)     drama   

                                         DESCRIPTION  
0  Listening in to a conversation betwee

## Data PreProcessing

In [7]:

# text features
train_data_df['TEXT'] = train_data_df['TITLE'] + ' ' + train_data_df['DESCRIPTION']
test_data_solution_df['TEXT'] = test_data_solution_df['TITLE'] + ' ' + test_data_solution_df['DESCRIPTION']

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
    
train_data_df['TEXT'] = train_data_df['TEXT'].apply(preprocess_text)
test_data_solution_df['TEXT'] = test_data_solution_df['TEXT'].apply(preprocess_text)

X_train, X_val, y_train, y_val = train_test_split(train_data_df['TEXT'], train_data_df['GENRE'], test_size=0.2, random_state=42)


## Feature Extraction

In [8]:
vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(test_data_solution_df['TEXT'])

## Logestic Regression

In [11]:


lr_params = {'C': [0.1, 1, 10, 100]}
lr_classifier = GridSearchCV(LogisticRegression(max_iter=1000), lr_params, cv=5)
lr_classifier.fit(X_train_tfidf, y_train)

y_val_pred_lr = lr_classifier.predict(X_val_tfidf)
y_test_pred_lr = lr_classifier.predict(X_test_tfidf)
print("Logistic Regression Validation Set Evaluation:")
print(classification_report(y_val, y_val_pred_lr))
print("Logistic Regression Validation Accuracy:", accuracy_score(y_val, y_val_pred_lr))
print("\nLogistic Regression Test Set Evaluation:")
print(classification_report(test_data_solution_df['GENRE'], y_test_pred_lr))
print("Logistic Regression Test Accuracy:", accuracy_score(test_data_solution_df['GENRE'], y_test_pred_lr))


C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

Logistic Regression Validation Set Evaluation:


C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      action       0.52      0.27      0.36       263
       adult       0.69      0.21      0.33       112
   adventure       0.46      0.18      0.26       139
   animation       0.47      0.08      0.13       104
   biography       0.00      0.00      0.00        61
      comedy       0.52      0.60      0.55      1443
       crime       0.30      0.03      0.05       107
 documentary       0.67      0.84      0.74      2659
       drama       0.54      0.77      0.64      2697
      family       0.44      0.10      0.16       150
     fantasy       0.00      0.00      0.00        74
   game-show       0.90      0.45      0.60        40
     history       0.00      0.00      0.00        45
      horror       0.65      0.59      0.62       431
       music       0.61      0.52      0.56       144
     musical       0.50      0.04      0.07        50
     mystery       0.67      0.04      0.07        56
        news       0.50    

C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      action       0.50      0.29      0.37      1314
       adult       0.63      0.22      0.33       590
   adventure       0.61      0.19      0.29       775
   animation       0.50      0.06      0.10       498
   biography       0.00      0.00      0.00       264
      comedy       0.53      0.59      0.56      7446
       crime       0.36      0.03      0.06       505
 documentary       0.66      0.84      0.74     13096
       drama       0.54      0.77      0.63     13612
      family       0.46      0.09      0.15       783
     fantasy       0.53      0.03      0.06       322
   game-show       0.89      0.51      0.65       193
     history       0.00      0.00      0.00       243
      horror       0.66      0.59      0.62      2204
       music       0.66      0.44      0.53       731
     musical       0.16      0.01      0.03       276
     mystery       0.38      0.02      0.03       318
        news       0.79    

C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## NaiveBayes

In [12]:

nb_params = {'alpha': [0.1, 0.5, 1, 5, 10]}
nb_classifier = GridSearchCV(MultinomialNB(), nb_params, cv=5)
nb_classifier.fit(X_train_tfidf, y_train)

y_val_pred_nb = nb_classifier.predict(X_val_tfidf)
y_test_pred_nb = nb_classifier.predict(X_test_tfidf)
print("\nNaive Bayes Validation Set Evaluation:")
print(classification_report(y_val, y_val_pred_nb))
print("Naive Bayes Validation Accuracy:", accuracy_score(y_val, y_val_pred_nb))
print("\nNaive Bayes Test Set Evaluation:")
print(classification_report(test_data_solution_df['GENRE'], y_test_pred_nb))
print("Naive Bayes Test Accuracy:", accuracy_score(test_data_solution_df['GENRE'], y_test_pred_nb))



Naive Bayes Validation Set Evaluation:


C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      action       0.59      0.22      0.32       263
       adult       0.74      0.22      0.34       112
   adventure       0.43      0.09      0.14       139
   animation       0.67      0.06      0.11       104
   biography       0.00      0.00      0.00        61
      comedy       0.51      0.47      0.49      1443
       crime       0.50      0.01      0.02       107
 documentary       0.60      0.86      0.70      2659
       drama       0.48      0.80      0.60      2697
      family       0.44      0.03      0.05       150
     fantasy       0.50      0.01      0.03        74
   game-show       1.00      0.42      0.60        40
     history       0.00      0.00      0.00        45
      horror       0.74      0.48      0.58       431
       music       0.66      0.45      0.53       144
     musical       0.00      0.00      0.00        50
     mystery       1.00      0.02      0.04        56
        news       0.00    

C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

      action       0.52      0.20      0.29      1314
       adult       0.54      0.18      0.27       590
   adventure       0.62      0.12      0.20       775
   animation       0.64      0.03      0.05       498
   biography       0.00      0.00      0.00       264
      comedy       0.53      0.46      0.49      7446
       crime       0.18      0.00      0.01       505
 documentary       0.59      0.85      0.70     13096
       drama       0.48      0.81      0.60     13612
      family       0.67      0.03      0.06       783
     fantasy       0.58      0.05      0.09       322
   game-show       0.91      0.52      0.66       193
     history       0.00      0.00      0.00       243
      horror       0.69      0.45      0.54      2204
       music       0.64      0.44      0.52       731
     musical       0.00      0.00      0.00       276
     mystery       0.33      0.00      0.01       318
        news       1.00    

C:\Users\amnak\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
